In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
from torchvision.datasets import CIFAR10
from torchvision.transforms import Compose, ToTensor, Normalize, Resize, RandomHorizontalFlip, RandomCrop
from torch.utils.data import DataLoader
from torchvision.models import resnet18
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision.datasets import CIFAR10
from torchvision.transforms import Compose, ToTensor, Normalize, Resize, RandomHorizontalFlip, RandomCrop
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

#可变线性层+升维马尔科夫转移过程
class AdaptiveWeightsNet(nn.Module):
    def __init__(self, feature_dim, num_classes):
        super(AdaptiveWeightsNet, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(feature_dim, feature_dim // 2),
            nn.ReLU(),
            nn.Linear(feature_dim // 2, num_classes),
            nn.Softmax(dim=1)
        )
    
    def forward(self, features):
        weights = self.mlp(features)
        return weights
    
class DistanceMetric(nn.Module):
    def __init__(self, feature_dim):
        super(DistanceMetric, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(feature_dim * 2, feature_dim),
            nn.ReLU(),
            nn.Linear(feature_dim, 1),
            nn.ReLU()
        )
    
    def forward(self, attractor1, attractor2):
        combined_features = torch.cat((attractor1, attractor2), dim=1)
        distance = self.mlp(combined_features)
        return distance

class ModifiedFeatureExtractor(nn.Module):
    def __init__(self, pretrained=True):
        super(ModifiedFeatureExtractor, self).__init__()
        self.model = resnet18(pretrained=pretrained)
        self.pretrained = pretrained

    def forward(self, x):
        if self.pretrained:
            # 在预训练模式下,使用完整的模型
            x = self.model(x)
        else:
            # 在特征提取模式下,使用除去最后一层的模型
            x = nn.Sequential(*list(self.model.children())[:-1])(x)
            x = torch.flatten(x, 1)
        return x

    def remove_last_layer(self):
        # 移除最后一层,用于特征提取
        self.pretrained = False

class DynamicResidualFeatureAggregationNetwork(nn.Module):
    def __init__(self, num_classes, feature_dim, high_dim):
        super(DynamicResidualFeatureAggregationNetwork, self).__init__()
        self.attractors = nn.Parameter(torch.zeros(num_classes, high_dim))
        nn.init.kaiming_normal_(self.attractors, mode='fan_out', nonlinearity='relu')

        self.linear1 = nn.Linear(feature_dim, high_dim)
        self.linear2 = nn.Linear(high_dim, feature_dim)

        self.mlp = nn.Sequential(
            nn.Linear(feature_dim, feature_dim // 2),
            nn.ReLU(inplace=True),
            nn.Linear(feature_dim // 2, num_classes)
        )

        self.distance_metric = DistanceMetric(high_dim)
        self.adaptive_weights_net = AdaptiveWeightsNet(high_dim, num_classes)

    def forward(self, features):
        features_high = self.linear1(features)
        adaptive_weights = self.adaptive_weights_net(features_high)
        weighted_attractors = self.attractors * adaptive_weights.unsqueeze(2)
        aggregated_features_high = torch.bmm(adaptive_weights.unsqueeze(1), weighted_attractors).squeeze(1)
        aggregated_features = self.linear2(aggregated_features_high)
        combined_features = features + aggregated_features
        processed_features = self.mlp(combined_features)
        return processed_features

    def regularize_attractors(self):
        reg_loss = 0
        num_attractors = self.attractors.size(0)
        for i in range(num_attractors - 1):
            for j in range(i + 1, num_attractors):
                distance = self.distance_metric(self.attractors[i:i+1], self.attractors[j:j+1])
                reg_loss += distance.squeeze()
        reg_loss /= (num_attractors * (num_attractors - 1) / 2)
        return reg_loss

    def calculate_intra_class_compactness(self, features, labels):
        """计算类内紧致度"""
        unique_labels = torch.unique(labels)
        compactness = 0.0
        for label in unique_labels:
            class_features = features[labels == label]
            class_mean = torch.mean(class_features, dim=0, keepdim=True)
            compactness += torch.mean(torch.norm(class_features - class_mean, dim=1)) / len(unique_labels)
        return compactness
    
    def calculate_inter_class_distance(self, features, labels):
        """计算类间距离(使用余弦相似度)"""
        unique_labels = torch.unique(labels)
        class_centers = []
        for label in unique_labels:
            class_features = features[labels == label]
            class_center = torch.mean(class_features, dim=0, keepdim=True)
            class_centers.append(class_center)
        class_centers = torch.cat(class_centers, dim=0)

        # 计算类中心之间的余弦相似度
        distance = 0.0
        n = len(class_centers)
        for i in range(n):
            for j in range(i + 1, n):
                cos_sim = F.cosine_similarity(class_centers[i], class_centers[j], dim=0)
                distance += (1 - cos_sim) / (n * (n - 1) / 2)  # 使用1-cos_sim来转换相似度为距离
        return distance

class LearningModelTrainer:
    def __init__(self, feature_extractor, attention_attractor_net, config):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.feature_extractor = feature_extractor.to(self.device)
        self.attention_attractor_net = attention_attractor_net.to(self.device)
        self.config = config

    def pretrain_feature_extractor(self, loader):
        # 预训练特征提取器,此时的特征提取器包含全连接层
        optimizer = optim.SGD(self.feature_extractor.parameters(), lr=self.config["lr"], momentum=0.9)
        criterion = nn.CrossEntropyLoss()
        
        self.feature_extractor.train()
        for epoch in range(self.config["pretrain_epochs"]):
            for i, (images, labels) in enumerate(loader):
                images, labels = images.to(self.device), labels.to(self.device)
                optimizer.zero_grad()
                outputs = self.feature_extractor(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                if (i + 1) % 100 == 0:
                    print(f"Pretrain Epoch [{epoch+1}/{self.config['pretrain_epochs']}], Step [{i+1}/{len(loader)}], Loss: {loss.item():.4f}")

    def train_attractor_network(self, loader):
        self.feature_extractor.eval()  # 设置特征提取器为评估模式
        optimizer = optim.AdamW(self.attention_attractor_net.parameters(), lr=self.config["meta_lr"])
        criterion = nn.CrossEntropyLoss()
        lambda_reg = self.config["lambda_reg"]

        for epoch in range(self.config["attraction_epochs"]):
            total_loss_accumulated = 0  # 累积损失,用于日志
            for j, (images, labels) in enumerate(loader):
                images, labels = images.to(self.device), labels.to(self.device)
            
                # 获取特征表示
                with torch.no_grad():  # 不更新特征提取器的梯度
                    features = self.feature_extractor(images)
            
                # 计算吸引子网络的输出和分类损失
                optimizer.zero_grad()
                outputs = self.attention_attractor_net(features)
                classification_loss = criterion(outputs, labels)
            
                # 计算正则化损失
                reg_loss = self.attention_attractor_net.regularize_attractors()

                # 计算总损失并进行反向传播
                total_loss = classification_loss + lambda_reg * reg_loss
                total_loss.backward()
                optimizer.step()

                total_loss_accumulated += total_loss.item()

                if (j + 1) % 100 == 0:
                    avg_loss = total_loss_accumulated / 100
                    print(f"Attraction Epoch [{epoch+1}/{self.config['attraction_epochs']}], Step [{j+1}/{len(loader)}], Avg Loss over last 100 steps: {avg_loss:.4f}")
                    total_loss_accumulated = 0  # 重置累积损失

    def evaluate_accuracy(self, loader):
        self.feature_extractor.eval()
        self.attention_attractor_net.eval()
        correct = 0
        total = 0

        with torch.no_grad():
            for images, labels in loader:
                images, labels = images.to(self.device), labels.to(self.device)
                features = self.feature_extractor(images)
                outputs = self.attention_attractor_net(features)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = correct / total
        print(f'Accuracy on the test dataset: {accuracy * 100:.2f}%')

    def visualize_features(self, loader):
        self.feature_extractor.eval()
        self.attention_attractor_net.eval()
        
        features_list = []
        labels_list = []
        
        with torch.no_grad():
            for images, labels in loader:
                images, labels = images.to(self.device), labels.to(self.device)
                features = self.feature_extractor(images)
                features_list.append(features.cpu().numpy())
                labels_list.append(labels.cpu().numpy())
                
        # Concatenate lists to form arrays
        features_array = np.concatenate(features_list, axis=0)
        labels_array = np.concatenate(labels_list, axis=0)
        
        # Apply t-SNE
        tsne = TSNE(n_components=2, random_state=123)
        reduced_features = tsne.fit_transform(features_array)
        
        # Visualize
        plt.figure(figsize=(10, 10))
        colors = plt.cm.winter(np.linspace(0, 1, len(np.unique(labels_array))))  # Use the 'winter' colormap

        for i, color in zip(np.unique(labels_array), colors):
            indexes = labels_array == i
            plt.scatter(reduced_features[indexes, 0], reduced_features[indexes, 1], label=i, color=color)

        plt.legend()
        plt.title("Features Visualization using t-SNE")
        plt.grid(True)  # Add a grid for a more standard look
        plt.savefig("features_tsne_visualization.png")

    def evaluate_metrics(self, loader):
        self.feature_extractor.eval()
        self.attention_attractor_net.eval()
        features_list = []
        labels_list = []
        
        with torch.no_grad():
            for images, labels in loader:
                images, labels = images.to(self.device), labels.to(self.device)
                features = self.feature_extractor(images)
                features_list.append(features)
                labels_list.append(labels)
                
        features = torch.cat(features_list, dim=0)
        labels = torch.cat(labels_list, dim=0)
        
        # 计算并打印类内紧致度和类间距离
        compactness = self.attention_attractor_net.calculate_intra_class_compactness(features, labels)
        inter_class_distance = self.attention_attractor_net.calculate_inter_class_distance(features, labels)
        print(f"Class Intra-Compactness: {compactness.item():.4f}")
        print(f"Inter-Class Distance: {inter_class_distance.item():.4f}")
        
if __name__ == "__main__":
    transform = Compose([
        Resize((224, 224)),
        ToTensor(),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
    test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    feature_extractor = ModifiedFeatureExtractor(pretrained=True)
    attention_attractor_net = DynamicResidualFeatureAggregationNetwork(num_classes=10, feature_dim=512, high_dim=1024)  

    config = {
        "lr": 0.01,
        "pretrain_epochs": 3,
        "attraction_epochs": 2,
        "batch_size": 32,
        "meta_lr": 0.001,
        "lambda_reg": 0.001,
    }

    trainer = LearningModelTrainer(feature_extractor, attention_attractor_net, config)

    print("开始预训练特征提取器...")
    trainer.pretrain_feature_extractor(train_loader)

    feature_extractor.remove_last_layer()

    print("开始训练吸引子网络...")
    trainer.train_attractor_network(train_loader)  # 注意这里使用的是train_loader

    print("评估模型性能...")
    trainer.evaluate_accuracy(test_loader)  # 注意这里使用的是test_loader
    
    print("可视化特征...")
    trainer.visualize_features(test_loader)
    trainer.evaluate_metrics(test_loader)